In [ ]:
import polars as pl
from dotenv import load_dotenv
from libraries.client_stashapp import StashAppClient, get_stashapp_client

load_dotenv()

stash_client = StashAppClient()
stash_raw_client = get_stashapp_client()

all_stash_studios = stash_client.get_studios()
all_stashapp_performers = stash_client.get_performers()

use_studio_code_tag = stash_raw_client.find_tag("Filenames: Use Studio Code")

# Lookup functions
def get_by_parent_studio(parent_studio_id):
    return all_stash_studios.filter(pl.col("stash_studios_parent_studio_id") == parent_studio_id)

def get_studio(studio_id):
    return all_stash_studios.filter(pl.col("stash_studios_id") == studio_id)

def get_studio_by_name(studio_name):
    return all_stash_studios.filter(pl.col("stash_studios_name") == studio_name)

def get_performer(performer_id):
    return all_stashapp_performers.filter(pl.col("stashapp_id") == performer_id)

In [ ]:
# Scan the videos
videos_paths = ["W:\\Culture\\Videos", "X:\\Culture\\Videos", "Y:\\Culture\\Videos", "Z:\\Culture\\Videos"]

stash_raw_client.metadata_scan(videos_paths, {
    "scanGenerateClipPreviews": False,
    "scanGenerateCovers": True,
    "scanGenerateImagePreviews": False,
    "scanGeneratePhashes": True,
    "scanGeneratePreviews": False,
    "scanGenerateSprites": False,
    "scanGenerateThumbnails": False
})

In [ ]:
# current_studios = get_by_parent_studio("999")
current_studios = get_studio_by_name("Blacked")
current_studio_ids = current_studios.select(pl.col("stash_studios_id")).to_series().to_list()

current_studios

# Rename scenes

In [ ]:
scenes_df = stash_client.find_scenes_by_studio(current_studio_ids)
print(len(scenes_df))

In [ ]:
import os
from libraries.file_renamer import create_filename

scene_renames_df = scenes_df.select([
    pl.col("stashapp_id"),
    pl.col("stashapp_primary_file_path").map_elements(lambda directory: os.path.dirname(directory), return_dtype=pl.Utf8).alias("directory"),
    pl.col("stashapp_primary_file_basename").alias("old_filename"),
    pl.struct(["*"]).map_elements(lambda row: create_filename(use_studio_code_tag, row), return_dtype=pl.Utf8).alias("new_filename"),
    pl.col("stashapp_primary_file_path").alias("old_path"),
]).with_columns([
    pl.col("new_filename").map_elements(lambda filename: len(filename) if filename else 0, return_dtype=pl.Int32).alias("new_filename_length"),
    pl.concat_str([
        pl.col("directory"),
        pl.lit(os.sep),
        pl.col("new_filename")
    ]).alias("new_path")
])

# Reorder the columns
scene_renames_df = scene_renames_df.select([
    pl.col("stashapp_id"),
    pl.col("directory"),
    pl.col("old_filename"),
    pl.col("new_filename"),
    pl.col("new_filename_length"),
    pl.col("old_path"),
    pl.col("new_path"),
])

scene_renames_df

In [ ]:
# Rename the files
import os
import polars as pl

scenes_success_rows = []
scenes_failed_rows = []

for row in scene_renames_df.iter_rows(named=True):
    old_path = row['old_path']
    new_path = row['new_path']
    failure_info = dict(row)  # Create a copy of the row data

    # Check for missing hash or new path
    if new_path is None:
        failure_info['failure_reason'] = "Missing new path"
        failure_info['error_message'] = f"New path not found for {old_path}"
        scenes_failed_rows.append(failure_info)
        print(failure_info['error_message'])
        continue

    # Attempt to move the file if the old path is a file and the new path does not exist
    if os.path.isfile(old_path):
        if not os.path.exists(new_path):
            try:
                os.rename(old_path, new_path)

                # Check for and rename AI JSON sidecar file if it exists
                ai_json_path = f"{old_path}.AI.json"
                if os.path.isfile(ai_json_path):
                    new_ai_json_path = f"{new_path}.AI.json"
                    try:
                        os.rename(ai_json_path, new_ai_json_path)
                        print(f"Renamed AI JSON sidecar file:\n{ai_json_path}\n{new_ai_json_path}")
                    except Exception as e:
                        failure_info['failure_reason'] = "Sidecar rename failed"
                        failure_info['error_message'] = str(e)
                        scenes_failed_rows.append(failure_info)
                        continue
                print(f"Rename file:\n{old_path}\n{new_path}\n")
                scenes_success_rows.append(row)
            except Exception as e:
                failure_info['failure_reason'] = "Rename failed"
                failure_info['error_message'] = str(e)
                scenes_failed_rows.append(failure_info)
                print(f"Failed to rename:\n{old_path}\n{new_path}\n{e}")
        else:
            failure_info['failure_reason'] = "File exists"
            failure_info['error_message'] = f"A file already exists in the new path: {new_path}"
            scenes_failed_rows.append(failure_info)
            print(failure_info['error_message'])
    else:
        failure_info['failure_reason'] = "File not found"
        failure_info['error_message'] = f"File does not exist: {old_path}"
        scenes_failed_rows.append(failure_info)
        print(failure_info['error_message'])

scenes_success_df = pl.DataFrame(scenes_success_rows) if scenes_success_rows else pl.DataFrame(schema=scene_renames_df.schema)
scenes_failed_df = pl.DataFrame(scenes_failed_rows) if scenes_failed_rows else pl.DataFrame(schema={**scene_renames_df.schema, 'failure_reason': pl.Utf8, 'error_message': pl.Utf8})

In [ ]:
scenes_success_df

In [ ]:
scenes_failed_df